# Manipulação de Imagens RGB

Our image pre-processing consists in the following steps:
a) Image conversion to gray scale
b) Flatenning 2D->1D conversion


## **1. Importações**

In [8]:
%matplotlib inline
#pip install -U scikit-learn
#pip install imagehash
#pip install -U matplotlib

import sklearn

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

from sklearn.feature_selection import SequentialFeatureSelector

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

from sklearn.cluster import DBSCAN, KMeans

from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import  classification_report

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier

from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier

from sklearn.neural_network import MLPClassifier

from sklearn.naive_bayes import GaussianNB

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler

import seaborn as sns
sns.set()

from PIL import Image

import imagehash

import numpy as np
from numpy import mean
from numpy import isnan
from numpy import asarray
from numpy import polyfit

import statistics

import os
import shutil, glob, os.path

from random import shuffle

from tqdm import tqdm

from scipy.stats import pearsonr

import math

import skimage
from skimage import data, io, color
from skimage.filters import threshold_otsu 
from skimage.feature import canny 


## **2. Leitura das imagens na pasta**

In [23]:
#contagem e verificação das imagens

INPUT_PATH = '/home/samuel/Downloads/AvaliaçãoPrática/python-kmeans/faces'

Files=os.listdir(INPUT_PATH)
print("files: ", os.listdir(INPUT_PATH))
nImg=len(Files)

print("number of images: ",nImg)

for filename in Files:
    if filename.endswith('.jpg'):
        try:
            img = Image.open(INPUT_PATH+"/"+filename) # open the image file
            img.verify() # verify that it is, in fact an image
        except (IOError, SyntaxError) as e:
            print('Bad file:', filename)
            #os.remove(base_dir+"\\"+filename) (Maybe)

files:  ['10x10.jpg']
number of images:  1


## **3.**

In [ ]:
# preprocessamento das imagens do arquivo -> retorna uma imagem 1D em grayscale normalizada entre 0:255

def image_loader(path,toplt,topltgray):
    Image = io.imread(path) # load image in np.array
    if toplt:
        fig2, ax2 = plt.subplots(1,2, figsize=(12, 5.5))
        ax2[0].imshow(Image)
        ax2[1].hist(Image.ravel(), bins = 64, color = 'orange', alpha = 0.5)
        ax2[1].hist(Image[:, :, 0].ravel(), bins = 64, color = 'red', alpha = 0.5)
        ax2[1].hist(Image[:, :, 1].ravel(), bins = 64, color = 'Green', alpha = 0.5)
        ax2[1].hist(Image[:, :, 2].ravel(), bins = 64, color = 'Blue', alpha = 0.5)
        ax2[1].set_xlim(0, 255)
        ax2[1].set_xlabel('Intensity Value')
        ax2[1].set_ylabel('Count')
        ax2[1].legend(['Total', 'Red_Channel', 'Green_Channel', 'Blue_Channel'])
        plt.show()
    image=[] # vetor 1D de pixels monocromáticos
    # debb
    if topltgray:
        grayimg=np.zeros((Image.shape[0],Image.shape[1]),dtype=float)
    
    if Image.shape[2]>=3: # if RGB or similar
        for i in range(Image.shape[0]):
            for j in range(Image.shape[1]):
                image.append(int(0.2989*Image[i][j][0]+0.5870*Image[i][j][1]+0.1140*Image[i][j][2])) # color to gray scale
                # debb
                if topltgray:
                    grayimg[i][j]=0.2989*Image[i][j][0]+0.5870*Image[i][j][1]+0.1140*Image[i][j][2]
        #debb  
        if topltgray:
            Mx=np.max(grayimg)
            mn=np.min(grayimg)
            for i in range(Image.shape[0]):
                for j in range(Image.shape[1]):
                    grayimg[i][j]=int(255*(grayimg[i][j]-mn)/(Mx-mn))
            fig2, ax2 = plt.subplots(1,2, figsize=(12, 5.5))
            ax2[0].imshow(grayimg)
            ax2[1].hist(grayimg.ravel(), bins = 64, color = 'orange',alpha = 0.5)
            ax2[1].set_xlim(0, 255)
            ax2[1].set_xlabel('Intensity Value')
            ax2[1].set_ylabel('Count')
            ax2[1].legend(['Total'])
            plt.show()
    else: # gray inpt image
        image.append(int(Image[i][j]))
        
    # normalization of grayscale to 0:255 interval for each image (MinMaxScaler)
    Mx=np.max(image)
    mn=np.min(image)
    for i in range(len(image)):
        image[i]=int(255*(image[i]-mn)/(Mx-mn))
        
    return Image.shape[0], Image.shape[1], np.array(image)


## **4.**

In [ ]:
# load and preprocess image data set + MAPEA O INTERVALO DE VARIAÇÃO DO TAMANHO n DAS IMAGENS NA PASTA 

X=[]
ResX=[]
ResY=[]
n=[]

for file in tqdm(Files[:nImg]): # tqdm(Files[:4]): # 
    if file.endswith('.jpg'):
      path=os.path.join(INPUT_PATH, file)
      X.append([])
      
      print(INPUT_PATH+'/'+file)
      RX, RY, Img = image_loader(path,True,True) # returns a grayscale image of the same size. bool prm: plot RGB & plot grayscale
      
      n.append(Img.shape[0])
      
      print(file," ",RX,"x",RY," n = ",n[-1])
      
      ResX.append(RX)
      ResY.append(RY)
      X[-1].append(Img)

# PLOTA HISTOGRAMA DE TAMANHOS
plt.hist(n,bins=100)
plt.show()
print("menor imagem ",np.min(n)," maior imagem ",np.max(n))

In [ ]:
# Y contains the images to process

nSamples=len(X)
Y=[]

print("nSamples: ",nSamples)
for i in range(nSamples):
    #debb
    #print('sample ', i)    
    Y.append([])
    Y[-1]=X[i][0]
    #debb
    #print(np.min(Y[-1]),":",np.max(Y[-1]))

#debb
#print(Y)

## **5. Implementação do K-Means**

In [26]:
from random import random
from datetime import datetime

RED = 0
GREEN = 1
BLUE = 2
INF = 1e10
v = []
m = []

# implementa o algoritmo k-means
def kmeans(vetor, k_clusters):
  tamanho = len(vetor)
  print("Tamanho do vetor: ", tamanho)
  # inicializa vetor de classificacao
  global classificacao 
  classificacao = [-1 for i in range(tamanho)]
  red_cluster = [-1 for i in range(k_clusters)]
  green_cluster = [-1 for i in range(k_clusters)]
  blue_cluster = [-1 for i in range(k_clusters)]
  m = [0 for i in range(k_clusters)]
  
  # define clusters aleatoriamente
  for k in range(k_clusters):
    posicao = round(tamanho*random())
    red_cluster[k] = vetor[posicao][RED]
    green_cluster[k] = vetor[posicao][GREEN]
    blue_cluster[k] = vetor[posicao][BLUE]
  iteracoes = 0
  # inicio das iterações
  tempo_inicio = datetime.now()
  while True:
    iteracoes+=1
    # imprime o numero de iterações a cada 10
    # if iteracoes % 10 == 0:
    #   print("ITERAÇÕES: ", iteracoes)
    mudanca = 0
    for i in range(tamanho):
      distancia_minima = INF
      k_menor = -INF
      for k in range(k_clusters):
        distancia = round(math.sqrt((int(vetor[i][RED])-int(red_cluster[k]))**2+(int(vetor[i][GREEN])-int(green_cluster[k]))**2+(int(vetor[i][BLUE])-int(blue_cluster[k]))**2)) # forma L2 para calcular a distância do RGB do pixel em relação ao RGB do cluster
        if distancia < distancia_minima:
          #print("DISTANCIA: ", distancia_minima, distancia)
          distancia_minima = distancia
          k_menor = k
      if mudanca == 0 and k_menor != -INF and classificacao[i] != k_menor:
        classificacao[i] = k_menor # define o cluster a que pertence o pixel
        #print("CLASSIFICACAO: ", classificacao[i], "i: ", i, "K: ", k, "DISTANCIA_MINIMA", distancia_minima)
        mudanca = 1
    if mudanca == 0:
      print("CONVERGENCIA OBTIDA!")
      # print("ITERAÇÕES: ", iteracoes)
      # print("CLASSIFICAÇÃO", classificacao)
      break
    else:
      if mudanca > 0:
        for k in range(k_clusters):
          red_cluster[k] = 0
          green_cluster[k] = 0
          blue_cluster[k] = 0
          m[k] = 0 # contador de pixels por clusters
          for i in range(tamanho):
            if classificacao[i] == k:
              red_cluster[k] += vetor[i][RED]
              green_cluster[k] += vetor[i][GREEN]
              blue_cluster[k] += vetor[i][BLUE]
              m[k] += 1
          if m[k] != 0:
            red_cluster[k] = round(red_cluster[k]/m[k])
            green_cluster[k] = round(green_cluster[k]/m[k])
            blue_cluster[k] = round(blue_cluster[k]/m[k])
            # print("K: ", k)
            # print("IMPRESSÃO CLUSTER: ", m[k], red_cluster[k], green_cluster[k], blue_cluster[k], "DIVISAO: ", red_cluster[k], green_cluster[k], blue_cluster[k])
  tempo_fim = datetime.now()
  global tempo_execucao
  tempo_execucao = calcular_tempo(tempo_inicio, tempo_fim)
  print("Tempo após cálculo: ", tempo_execucao)
  return iteracoes


# calcula o tempo de execução do algoritmo
def calcular_tempo(tempo_inicio, tempo_fim):
  resultado = tempo_fim-tempo_inicio
  return resultado

# lê a imagem e copia valores dos pixels para a lista v
def processar_imagem(path):
  imagem = io.imread(path)
  print("Tamanho da imagem: ", imagem.shape[0], " x ", imagem.shape[1])
  for i in range(imagem.shape[0]):
    for j in range(imagem.shape[1]):
      v.append([imagem[i][j][0], imagem[i][j][1], imagem[i][j][2]])
  # salva a leitura da imagem num arquivo de output
  # with open('./output.txt', 'a') as filehandle:
  #   filehandle.write(str(v))

# função para teste com uma imagem
def init_process(file):
  for file in Files[:1]: # tqdm(Files[:4]): # 
    if file.endswith('.jpg'):
      path=os.path.join(INPUT_PATH, file)
      processar_imagem(path)

#init_process()
#kmeans(v, 3)

## **6. Teste do Kmeans**
### **Instruções:**
#### Antes de executar os testes pela primeira vez, é necessário rodar os blocos 1, 2 e 5. Depois da primeira execução, pode-se executar os testes diretamente.
### Sobre os resultados:
#### A classificação dos pixels em cada rodada de testes será encontrada na pasta classificações. A nomeclatura utilizada é: class_nome-imagem_rodada_k-numero-clusters.txt. Ex.: 64x64_1_k-2.txt
#### Os tempos de execução serão encontrados na pasta raiz. A nomeclatura utilizda é: nome-imagem_numero-clusters.txt. Ex.: 64x64_2.txt
#### O arquivo de tempos de execução contém o tempo e ao lado, o número de iterações necessárias para se obter o resultado na rodada.

In [28]:
RODADAS_BENCHMARK = 20 # número de rodadas em que o kmeans será executado.
MIN_CLULSTERS = 2
MAX_CLUSTERS = 6 # ao definir o número máximo de clusters, somar + 1. Ex.: para execução com max 5 clusters, definir MAX_CLUSTERS como 5+1 (por conta do funcionamento da função range() do python).

def benchmark(n, k, nome_imagem): # n é o número de vezes que o kmeans será executado e k o número de clusters a ser passado para o método kmeans
  with open('./' + nome_imagem + '.txt', 'w') as filehandle:
    filehandle.write("")
  for i in range(n):
    total_iteracoes = kmeans(v, k)
    with open('./' + nome_imagem + '.txt', 'a') as filehandle:
      filehandle.write(str(tempo_execucao) + " - " + str(total_iteracoes))
      filehandle.write("\n")
    with open('./classificacoes/class_' + nome + '_' + str(i+1) + '_k-' + str(k) + '.txt', 'w') as filehandle:
      filehandle.write(str(classificacao))

for i in range(len(Files)): # Alterar para resolucao
  v.clear() 
  nome = str(Files[i].split('.')[0])
  path=os.path.join(INPUT_PATH, Files[i])
  processar_imagem(path)
  for k in range(MIN_CLULSTERS, MAX_CLUSTERS):
    benchmark(RODADAS_BENCHMARK, k, nome + '_' + str(k))


Tamanho da imagem:  10  x  10
Tamanho do vetor:  100
CONVERGENCIA OBTIDA!
Tempo após cálculo:  0:00:00.044101
Tamanho do vetor:  100
CONVERGENCIA OBTIDA!
Tempo após cálculo:  0:00:00.041227
Tamanho do vetor:  100
CONVERGENCIA OBTIDA!
Tempo após cálculo:  0:00:00.040405
Tamanho do vetor:  100
CONVERGENCIA OBTIDA!
Tempo após cálculo:  0:00:00.041099
Tamanho do vetor:  100
CONVERGENCIA OBTIDA!
Tempo após cálculo:  0:00:00.041734
Tamanho do vetor:  100
CONVERGENCIA OBTIDA!
Tempo após cálculo:  0:00:00.043740
Tamanho do vetor:  100
CONVERGENCIA OBTIDA!
Tempo após cálculo:  0:00:00.042630
Tamanho do vetor:  100
CONVERGENCIA OBTIDA!
Tempo após cálculo:  0:00:00.039293
Tamanho do vetor:  100
CONVERGENCIA OBTIDA!
Tempo após cálculo:  0:00:00.042307
Tamanho do vetor:  100
CONVERGENCIA OBTIDA!
Tempo após cálculo:  0:00:00.044587
Tamanho do vetor:  100
CONVERGENCIA OBTIDA!
Tempo após cálculo:  0:00:00.048032
Tamanho do vetor:  100
CONVERGENCIA OBTIDA!
Tempo após cálculo:  0:00:00.043469
Tamanho do